В этом файле: на частичной выборке датасета 0.15 сравниваем и определяем лучший метод из SVM, logreg, и randomforest. При этом извлекаем признаки из каждого изображения с помощью метода PCA.

1. Загружаем Dataset

In [ ]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("misrakahmed/vegetable-image-dataset")
path = os.path.join(path, 'Vegetable Images')
print("Path to dataset files:", path)

100%|██████████| 534M/534M [00:08<00:00, 62.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/misrakahmed/vegetable-image-dataset/versions/1/Vegetable Images


2. Преобразуем изображения в матрицу 64 х 64, извелкаем признаки с помощью PCA, преобразуем их в одномерныи вектор и объединяем (150 значении для каждого изображения), классифицируем с помощью логистической регрессии.

In [ ]:
import os
import cv2
import random
import numpy as np
from joblib import dump
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Путь к директориям
base_dir = path
categories = ["train", "test", "validation"]

def load_images_and_labels(base_dir, category, sample_percentage=0.15):
    """
    Функция загрузки изображений и соответствующих меток.

    :param base_dir: путь к корневой директории
    :param category: категория ('train', 'test', 'validation')
    :param sample_percentage: процент выборки данных
    :return: массивы изображений и меток
    """
    images = []
    labels = []
    path = os.path.join(base_dir, category)
    for class_name in tqdm(os.listdir(path)):
        class_dir = os.path.join(path, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Получаем список всех файлов в данной категории
        all_files = os.listdir(class_dir)

        # Выбираем случайным образом % файлов
        selected_files = random.sample(all_files, int(len(all_files) * sample_percentage))

        for img_name in tqdm(selected_files):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                images.append(img.flatten())
                labels.append(class_name)

    return np.array(images), np.array(labels)

# Загрузка данных
train_images, train_labels = load_images_and_labels(base_dir, "train", sample_percentage=0.15)
test_images, test_labels = load_images_and_labels(base_dir, "test", sample_percentage=0.15)

# Применение PCA для извлечения признаков
pca = PCA(n_components=150, svd_solver='randomized', whiten=True, random_state=42)
reduced_train_images = pca.fit_transform(train_images)
reduced_test_images = pca.transform(test_images)

# Классификация с использованием логистической регрессии
logreg = LogisticRegression(max_iter=500)

# Настройка параметров для поиска по сетке
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(logreg, param_grid, cv=5)

# Поиск по сетке и определение лучших параметров
grid.fit(reduced_train_images, train_labels)

# Прогнозирование на тестовом наборе
y_pred = grid.predict(reduced_test_images)

# Оценка точности
accuracy = accuracy_score(test_labels, y_pred)
print(f"Точность на тестовом наборе: {accuracy:.2f}")

# Вывод информации о значениях параметра C и их соответствующих оценках
cv_results = grid.cv_results_
for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    print(f"C: {params['C']}, Accuracy: {mean_score:.4f}")

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Точность на тестовом наборе: 0.29
C: 0.001, Accuracy: 0.2960
C: 0.01, Accuracy: 0.2876
C: 0.1, Accuracy: 0.2787
C: 1, Accuracy: 0.2720
C: 10, Accuracy: 0.2684
C: 100, Accuracy: 0.2680


3. Преобразуем изображения в матрицу 64 х 64, извелкаем признаки с помощью PCA, преобразуем их в одномерныи вектор и объединяем (150 значении для каждого изображения), классифицируем с помощью RandomForestClassifier.

In [ ]:
import os
import cv2
import random
import numpy as np
from joblib import dump
from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Путь к директориям
base_dir = path
categories = ["train", "test", "validation"]

def load_images_and_labels(base_dir, category, sample_percentage=0.15):
    """
    Функция загрузки изображений и соответствующих меток.

    :param base_dir: путь к корневой директории
    :param category: категория ('train', 'test', 'validation')
    :param sample_percentage: процент выборки данных
    :return: массивы изображений и меток
    """
    images = []
    labels = []
    path = os.path.join(base_dir, category)
    for class_name in tqdm(os.listdir(path)):
        class_dir = os.path.join(path, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Получаем список всех файлов в данной категории
        all_files = os.listdir(class_dir)

        # Выбираем случайным образом % файлов
        selected_files = random.sample(all_files, int(len(all_files) * sample_percentage))

        for img_name in tqdm(selected_files):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                images.append(img.flatten())
                labels.append(class_name)

    return np.array(images), np.array(labels)

# Загрузка данных
train_images, train_labels = load_images_and_labels(base_dir, "train", sample_percentage=0.15)
test_images, test_labels = load_images_and_labels(base_dir, "test", sample_percentage=0.15)

# Применение PCA для извлечения признаков
pca = PCA(n_components=150, svd_solver='randomized', whiten=True, random_state=42)
reduced_train_images = pca.fit_transform(train_images)
reduced_test_images = pca.transform(test_images)

# Классификация с использованием Random Forest
rfc = RandomForestClassifier(random_state=42)

# Настройка параметров для поиска по сетке
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2]
}
grid = GridSearchCV(rfc, param_grid, cv=5)

# Поиск по сетке и определение лучших параметров
grid.fit(reduced_train_images, train_labels)

# Прогнозирование на тестовом наборе
y_pred = grid.predict(reduced_test_images)

# Оценка точности
accuracy = accuracy_score(test_labels, y_pred)
print(f"Точность на тестовом наборе: {accuracy:.2f}")

# Вывод информации о значениях параметров и их соответствующих оценках
cv_results = grid.cv_results_
for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    print(f"Params: {params}, Accuracy: {mean_score:.4f}")

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Точность на тестовом наборе: 0.56
Params: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 100}, Accuracy: 0.4587
Params: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 200}, Accuracy: 0.4911
Params: {'max_depth': None, 'min_samples_leaf': 2, 'n_estimators': 100}, Accuracy: 0.4578
Params: {'max_depth': None, 'min_samples_leaf': 2, 'n_estimators': 200}, Accuracy: 0.4898
Params: {'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 100}, Accuracy: 0.3582
Params: {'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 200}, Accuracy: 0.3889
Params: {'max_depth': 5, 'min_samples_leaf': 2, 'n_estimators': 100}, Accuracy: 0.3631
Params: {'max_depth': 5, 'min_samples_leaf': 2, 'n_estimators': 200}, Accuracy: 0.3902
Params: {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 100}, Accuracy: 0.4160
Params: {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 200}, Accuracy: 0.4516
Params: {'max_depth': 10, 'min_samples_leaf': 2, 'n_estimators': 100}, Accuracy: 0

4. Преобразуем изображения в матрицу 64 х 64, извелкаем признаки с помощью PCA, преобразуем их в одномерныи вектор и объединяем (150 значении для каждого изображения), классифицируем с помощью SVM. (Выборка _%)

In [ ]:
import os
import cv2
import random
import numpy as np
from joblib import dump
from tqdm.auto import tqdm
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Путь к директориям
base_dir = path
categories = ["train", "test", "validation"]

def load_images_and_labels(base_dir, category, sample_percentage=0.15):
    """
    Функция загрузки изображений и соответствующих меток.

    :param base_dir: путь к корневой директории
    :param category: категория ('train', 'test', 'validation')
    :param sample_percentage: процент выборки данных
    :return: массивы изображений и меток
    """
    images = []
    labels = []
    path = os.path.join(base_dir, category)
    for class_name in tqdm(os.listdir(path)):
        class_dir = os.path.join(path, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Получаем список всех файлов в данной категории
        all_files = os.listdir(class_dir)

        # Выбираем случайным образом % файлов
        selected_files = random.sample(all_files, int(len(all_files) * sample_percentage))

        for img_name in tqdm(selected_files):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                images.append(img.flatten())
                labels.append(class_name)

    return np.array(images), np.array(labels)

# Загрузка данных
train_images, train_labels = load_images_and_labels(base_dir, "train", sample_percentage=0.15)
test_images, test_labels = load_images_and_labels(base_dir, "test", sample_percentage=0.15)

# Применение PCA для извлечения признаков
pca = PCA(n_components=150, svd_solver='randomized', whiten=True, random_state=42)
reduced_train_images = pca.fit_transform(train_images)
reduced_test_images = pca.transform(test_images)

# Классификация с использованием SVM
svc = SVC(kernel='rbf', class_weight='balanced')

# Настройка параметров для поиска по сетке
param_grid = {'C': [1, 5, 10, 50]}
grid = GridSearchCV(svc, param_grid, cv=5)

# Поиск по сетке и определение лучших параметров
grid.fit(reduced_train_images, train_labels)

# Прогнозирование на тестовом наборе
y_pred = grid.predict(reduced_test_images)

# Оценка точности
accuracy = accuracy_score(test_labels, y_pred)
print(f"Точность на тестовом наборе: {accuracy:.2f}")

# Вывод информации о значениях параметра C и их соответствующих оценках
cv_results = grid.cv_results_
for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    print(f"C: {params['C']}, Accuracy: {mean_score:.4f}")

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Точность на тестовом наборе: 0.48
C: 1, Accuracy: 0.4013
C: 5, Accuracy: 0.4093
C: 10, Accuracy: 0.4058
C: 50, Accuracy: 0.4062
